# CocktailDB API Raw

-  Reference: https://www.kaggle.com/ai-first/cocktail-ingredients/discussion/39910

In [1]:
import os
import sys
import bs4
import requests
import time
import numpy as np
import pandas as pd
import json
import urllib3
urllib3.disable_warnings()
from bs4 import BeautifulSoup as bs
from itertools import chain
from dask import bag
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options

In [3]:
#transform 500 JSON objects into a dataframe containing all the raw information
def get_random_drink():
    url = "https://www.thecocktaildb.com/api/json/v1/1/random.php"
    http = urllib3.PoolManager()
    response = http.request('GET', url)
    soup = bs(response.data,'html.parser')
    newDictionary=json.loads(str(soup))
    return list(chain(*newDictionary.values()))

drink_bag = bag.from_sequence(range(1000))
all_drinks = drink_bag.map(lambda x: get_random_drink()).compute()
drink_df = pd.DataFrame(list(chain(*all_drinks)))

#drink_df = drink_df[(drink_df.strCategory == 'Cocktail')|(drink_df.strCategory == 'Ordinary Drink')]

In [2]:
#eliminate the columns that we will not be using
useless_columns = ['dateModified','idDrink', 'strAlcoholic','strCategory','strDrinkThumb','strGlass','strIBA','strInstructions','strVideo']
drink_df.drop(useless_columns, axis=1, inplace=True)
#print(drink_df)

NameError: name 'drink_df' is not defined

In [25]:
#order the columns
column_names = ['strDrink']
for i in range(1,16):
    column_names.append('strIngredient'+str(i))
    column_names.append('strMeasure'+str(i))

#rearrange drink columns
drink_df = drink_df[column_names]

dict = drink_df.set_index('strDrink').T.to_dict('list')
#print(dict)


{'Egg Nog - Healthy': ['Egg', '1/2 cup ', 'Sugar', '3 tblsp ', 'Condensed milk', '13 oz skimmed ', 'Milk', '3/4 cup skimmed ', 'Vanilla extract', '1 tsp ', 'Rum', '1 tsp ', 'Nutmeg', '\n', '', '\n', '', '\n', '', '\n', '', '\n', '', '\n', '', '', '', '', '', ''], 'Brandy Flip': ['Brandy', '2 oz ', 'Egg', '1 whole ', 'Sugar', '1 tsp superfine ', 'Light cream', '1/2 oz ', 'Nutmeg', '1/8 tsp grated ', '', ' ', '', ' ', '', ' ', '', ' ', '', ' ', '', '', '', '', '', '', '', '', '', ''], 'Gin Smash': ['Gin', '2 oz ', 'Carbonated water', '1 oz ', 'Sugar', '1 cube ', 'Mint', '4 ', 'Orange', '1 slice ', 'Cherry', '1 ', '', ' ', '', ' ', '', ' ', '', ' ', '', ' ', '', '', '', '', '', '', '', ''], 'Zombie': ['Rum', '1 1/2 oz', 'Gold rum', '1 1/2 oz', '151 proof rum', '1 oz', 'Pernod', '1 tsp', 'Grenadine', '1 tsp', 'Lime Juice', '1 tsp', 'Angostura Bitters', '1 drop', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], 'Stone Sour': ['Apricot brandy',

/Users/Maggie/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
df = pd.DataFrame.from_dict(dict, orient="index")
df.to_csv("cocktaildb_raw.csv")